## Решение задачи бинарной классификации

### Логистическая регрессия

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
np.random.seed(42)

In [ ]:
data = pd.read_csv('bike_buyers_clean.csv')

In [ ]:
data.head()

# Обзор данных

Проверьте типы колонок в датасете

In [ ]:
# your code here

Колонки с бинарными качественными признаками (типа пола) замените на чиселки

In [ ]:
# your code here

Оставим в данных только числовые колонки.

In [ ]:
# your code here

Поделите данные на матрицу объект-признак и ответ

In [ ]:
# your code here

Проверим сбалансированность классов. Если классы не сбалансированы, мы не можем использовать accuracy

In [ ]:
# your code here

## Масштабирование числовых признаков

In [ ]:
data['Income']

In [ ]:
# Очень большие значения, а у нас есть Пол со значниями 0 и 1...
# Надо масштабировать!

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(data[['Income']])

Важные замечания:
- классы sklearn возвращают numpy arrays, а не pandas dataframe, что неудобно
- любое преобразование признаков в ML-задачах нужно обучать только на тренировочных данных

In [ ]:
# пример работы с пайплайном
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train) # алгоритм находит параметры: mean и sigma. Только к обучающим данным!

X_train_sc = scaler.transform(X_train) # нормирует: x->(x-mean)/sigma
X_test_sc = scaler.transform(X_test) # с помощью mean и sigma от тренировочных данных изменяем test data

In [ ]:
X_train_sc

In [ ]:
X_train = pd.DataFrame(X_train_sc, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_sc, index=X_test.index, columns=X_test.columns)

X_train.head()

In [ ]:
y_train.value_counts()

Обучим логистическую регрессию и посмотрим на качество модели.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

# Соберем сделанные преобразования данных в pipeline

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression())
    ])

In [ ]:
# пример работы с пайплайном

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(accuracy_score(y_test, y_pred))

## Интерпретация результатов

In [ ]:
model.coef_, model.intercept_

In [ ]:
pipe['model_'].coef_, pipe['model_'].intercept_

In [ ]:
coefs = pd.DataFrame(model.coef_, index=np.arange(len(model.coef_)), columns=X_train.columns)
coefs['Intercept'] = model.intercept_
coefs

$$ y = \sigma(0.38*Income - 0.16*Children - 0.53*Cars - 0.1*Age - 0.07) $$

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X, y, cv=3).mean()

## Задача распознавания лиц

Решим задачу классификации лиц (задача многоклассовой классификации) с помощью метода опорных векторов.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Нарисуем несколько лиц из датасета

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(9,9))
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

Каждое изображение имеет размер [62×47] - это примерно 3000 пикселей. Мы можем использовать пиксели как признаки, но давайте понизим размерность пространства признаков.

Извлечем 150 самых информативных признаков из данных фотографий методом RandomizedPCA (тот самый метод главных компонент).

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA #Principal Components Analysis
from sklearn.pipeline import make_pipeline, Pipeline

pca = PCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42)

Попробуем решить задачу алгоритмом SVM.

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

Соберем пайплайн из метода опорных векторов + метода главных компонент.

In [ ]:
svc = SVC(kernel='linear', class_weight='balanced')

model_svm = make_pipeline(pca, svc)

Погридсерчим параметр С. 

Как гридсерчить: нужно собрать сетку параметров (это словарь, где ключи - названия гиперпараметров, а значения - их возможные значения, которые мы перебираем), а потом пихнуть в гридсерч модель, сетку и возможно указать какие-то параметры серча (scoring, cv...).

Если гридсерчим пайплайн, то для каждого из шагов пайпа можно указывать параметры, тогда сперва указывается название шага, а потом через двойное нижнее подчеркивание название параметра. 

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50]}

grid = GridSearchCV(model_svm, param_grid)

%time grid.fit(Xtrain, ytrain)

print(grid.best_params_)

In [ ]:
def get_best_model(model, Xtrain, ytrain):
    model_pipe = Pipeline([('pca',pca),('model',model)])
    
    param_grid = {'model__C': [0.01, 0.1, 1, 5, 10]}
    
    grid = GridSearchCV(model_pipe, param_grid)
    
    grid.fit(Xtrain, ytrain)
    
    return grid.best_estimator_

In [ ]:
model = grid.best_estimator_

yfit = model.predict(Xtest)

In [ ]:
fig, ax = plt.subplots(4, 6, figsize=(9,9))
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

Выведем на экран метрики классификации.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

Нарисуем матрицу ошибок классификатора.

In [ ]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(8,8))
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

### Задание 1.

Решите задачу распознавания лиц с помощью SVM с ядром. Попробуйте различные ядра: 'poly', 'rbf', 'sigmoid'.

Подберите гиперпараметры по кросс-валидации. 

SVM с каким ядром дал лучший результат?

In [ ]:
#your code here

### Задание 2.

Решите задачу распознавания лиц с помощью логистической регрессии (она также поддерживает опцию class_weight='balanced'):

1) Объявите модель, состоящую из pipeline(pca,logistic regression)

2) Подберите по сетке параметр C логистической регрессии (с помощью GridSearch)

3) Обучите модель на тренировочных данных и выведите наилучшие параметры модели

4) KNN

5) Naive Bayes

Какое качество показала эта модель?

In [ ]:
from sklearn.linear_model import LogisticRegression

#your code here

lr = ...
model = make_pipeline(...)

param_grid = ...
grid = GridSearchCV(model, param_grid)

...